In [9]:
import datajoint as dj
from pynwb import NWBHDF5IO, NWBFile
import spyglass.common as sgc
from spyglass.common import Nwbfile, Session, TaskEpoch, IntervalList
from hex_maze_behavior import populate_all_hexmaze, HexMazeConfig, HexCentroids, HexMazeBlock

dj.config['shared_modules'] = ['hex_maze']
schema = dj.schema('hex_maze')

nwb_file_name = "IM-1478_20220725_.nwb"
nwb_file_name = "BraveLu20240622_.nwb"
key = {"nwb_file_name": nwb_file_name}

populate_all_hexmaze(nwb_file_name)

hex_centroids = HexCentroids.get_hex_centroids_dict_cm(key)
print(hex_centroids)

/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cach

{'1': (170.72, 137.12), '10': (139.04, 118.56), '11': (150.88, 100.48), '12': (117.12, 130.56), '13': (128.16, 112.32), '14': (139.84, 93.92), '15': (106.56, 135.84), '16': (117.12, 118.4), '17': (128.64, 100.16), '18': (140.32, 81.44), '19': (96.16, 129.76), '2': (106.88, 23.36), '20': (106.56, 112.0), '21': (117.76, 93.6), '22': (129.12, 74.24), '23': (86.08, 134.88), '24': (96.0, 117.92), '25': (106.4, 99.68), '26': (117.76, 80.64), '27': (129.12, 61.6), '28': (75.84, 128.48), '29': (85.6, 111.36), '3': (47.2, 132.32), '30': (95.84, 92.8), '31': (106.72, 73.92), '32': (118.24, 54.72), '33': (66.56, 133.44), '34': (75.52, 116.96), '35': (85.12, 99.04), '36': (95.68, 80.32), '37': (106.72, 61.28), '38': (118.08, 42.4), '39': (65.28, 110.24), '4': (159.84, 130.88), '40': (74.72, 92.8), '41': (84.96, 73.92), '42': (96.0, 55.2), '43': (56.0, 115.84), '44': (64.64, 98.56), '45': (74.72, 80.32), '46': (84.96, 61.28), '47': (96.32, 42.72), '48': (56.96, 127.36), '48_left': (56.8, 138.4), '4

In [7]:
from spyglass.common import Session

# I dropped all the tables, oopsie! Put them back for BraveLu

bravelu_sessions = Session() & {"subject_id": "BraveLu"}
nwb_file_names = bravelu_sessions.fetch("nwb_file_name")

for name in nwb_file_names:
    print(name)
    

BraveLu20240505_.nwb
BraveLu20240513_.nwb
BraveLu20240516_.nwb
BraveLu20240518_.nwb
BraveLu20240519_.nwb
BraveLu20240613_.nwb
BraveLu20240615_.nwb
BraveLu20240617_.nwb
BraveLu20240619_.nwb
BraveLu20240622_.nwb


In [ ]:
# Saving for now: How to get processed position from the PositionOutput table for an nwbfile! 

nwb_file_name = "IM-1478_20220725_.nwb"
key = {"nwb_file_name": nwb_file_name}

# Get all valid run epochs
run_epochs = (HexMazeBlock & {"nwb_file_name": key["nwb_file_name"]}).fetch('epoch')

for run_epoch in set(run_epochs):
    # Get all position output tables for the nwbfile and the interval list for this run epoch
    # There will be multiple if it was processed with multiple sets of position parameters
    interval_list_name = f"pos {run_epoch} valid times"
    position_output_merge_tables = (
        PositionOutput.merge_get_part({"nwb_file_name": key["nwb_file_name"], "interval_list_name": interval_list_name})
    )

    # Get position data for each position output table
    merge_ids = position_output_merge_tables.fetch("KEY")
    for merge_id in merge_ids: 
        # Use this merge id to restrict the PositionOutput table and fetch the position data
        position_info = (PositionOutput & merge_id).fetch1_dataframe()
        print(merge_id)
        display(position_info)

{'merge_id': UUID('eab73e2c-358d-5657-3cc7-abd6c3f8ff85')}


,video_frame_ind,position_x,position_y,orientation,velocity_x,velocity_y,speed
time,,,,,,,
0.311375,0,NaN,NaN,NaN,NaN,NaN,NaN
0.373525,1,NaN,NaN,NaN,NaN,NaN,NaN
0.440090,2,NaN,NaN,NaN,NaN,NaN,NaN
0.501664,3,35.717411,70.014091,-2.034775,NaN,NaN,NaN
0.565580,4,35.795239,69.651075,-2.151286,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5202.862919,78036,29.932767,116.948020,-0.330466,-1.673055,0.233980,1.689337
5202.943314,78037,29.916518,116.944210,-0.336374,-1.218761,0.129413,1.225613
5203.007520,78038,29.854352,116.958565,-0.350923,-0.450997,0.063959,0.455509


[18:27:55][WARNING] Spyglass: Upsampled position data, frame indices are invalid. Setting add_frame_ind=False


{'merge_id': UUID('d9340e97-103b-099a-ca4d-96f6d9340cdd')}


,position_x,position_y,orientation,velocity_x,velocity_y,speed
time,,,,,,
0.311375,NaN,NaN,NaN,NaN,NaN,NaN
0.313375,NaN,NaN,NaN,NaN,NaN,NaN
0.315375,NaN,NaN,NaN,NaN,NaN,NaN
0.317375,NaN,NaN,NaN,NaN,NaN,NaN
0.319375,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5203.127088,29.949850,116.951076,-0.352008,0.274199,0.015577,0.274641
5203.129088,29.952332,116.951419,-0.352791,0.275171,0.015592,0.275612
5203.131088,29.954814,116.951763,-0.353578,0.275929,0.015607,0.276370


In [1]:
from hex_maze_behavior import HexPosition, HexPositionSelection, HexMazeBlock, HexCentroids, populate_all_hex_position
from spyglass.position import PositionOutput


populate_all_hex_position()

[2025-05-06 20:59:34,117][INFO]: DataJoint 0.14.4 connected to scrater@lmf-db.cin.ucsf.edu:3306


Skipping {'nwb_file_name': 'BraveLu20240622_.nwb', 'interval_list_name': 'pos 1 valid times'}. Check that this session has data in the PositionOutput table
Skipping {'nwb_file_name': 'BraveLu20240622_.nwb', 'interval_list_name': 'pos 3 valid times'}. Check that this session has data in the PositionOutput table
Skipping {'nwb_file_name': 'BraveLu20240622_.nwb', 'interval_list_name': 'pos 5 valid times'}. Check that this session has data in the PositionOutput table
Making with key {'pos_merge_id': UUID('4eeb7432-969b-f856-3cfb-dd4456754ac4'), 'nwb_file_name': 'Toby20250316_.nwb', 'epoch': 1, 'block': 2}


/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cach

Making with key {'pos_merge_id': UUID('713b3445-c1da-79f1-4b8e-c5760e604d20'), 'nwb_file_name': 'Toby20250316_.nwb', 'epoch': 1, 'block': 2}


[21:01:23][WARNING] Spyglass: Upsampled position data, frame indices are invalid. Setting add_frame_ind=False
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrate

Making with key {'pos_merge_id': UUID('4eeb7432-969b-f856-3cfb-dd4456754ac4'), 'nwb_file_name': 'Toby20250316_.nwb', 'epoch': 1, 'block': 3}


/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cach

KeyboardInterrupt: 

In [2]:
from hex_maze_behavior import HexPosition, HexPositionSelection, HexMazeBlock, HexCentroids
from spyglass.position import PositionOutput


display(HexPositionSelection())

display(HexPosition())

pos_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,1
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,2
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,3
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,1
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,2
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,3


pos_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch,analysis_file_name name of the file,hex_assignment_object_id
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,1,IM-1478_20220725_UMT382KOJM.nwb,4241cba2-6d2a-400d-aaf9-6a376759086d
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,2,IM-1478_20220725_RQUTK6XGIZ.nwb,a312723d-a4d8-4b78-b8ba-b156c10629ab
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,3,IM-1478_20220725_144DT22AR2.nwb,12515ee6-c24b-44da-943f-5a062d4dbe6d
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,1,IM-1478_20220725_SW6U1R3V8V.nwb,5bbe3ae9-8cc9-456f-8340-42c330927523
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,2,IM-1478_20220725_LRQW6XIAWB.nwb,42974c06-09fd-4d3e-9655-becb79314cf7
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,3,IM-1478_20220725_PVVCPR6FEJ.nwb,d7daebdd-4970-47b9-8c41-04ed84bb9746


In [3]:
HexPosition().drop()

[2025-05-06 20:18:18,979][INFO]: `hex_maze`.`__hex_position` (0 tuples)
[2025-05-06 20:18:23,458][INFO]: Dropped table `hex_maze`.`__hex_position`
[2025-05-06 20:18:23,489][INFO]: Tables dropped. Restart kernel.


In [ ]:
all_valid_keys = get_valid_keys_for_hex_position_selection()
print(f"Found {len(all_valid_keys)} valid key combinations for hex assignment")
for key in all_valid_keys:
    print(key)

{'nwb_file_name': 'IM-1478_20220725_.nwb', 'epoch': 0, 'merge_id': UUID('eab73e2c-358d-5657-3cc7-abd6c3f8ff85')}
{'nwb_file_name': 'IM-1478_20220725_.nwb', 'epoch': 0, 'merge_id': UUID('d9340e97-103b-099a-ca4d-96f6d9340cdd')}


In [2]:
from hex_maze_behavior import HexPosition, HexPositionSelection

display(HexPositionSelection())
display(HexPosition())

pos_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch
0859decd-5062-ee9c-a51f-575538d091f1,Toby20250316_.nwb,3,1
0859decd-5062-ee9c-a51f-575538d091f1,Toby20250316_.nwb,3,2
0859decd-5062-ee9c-a51f-575538d091f1,Toby20250316_.nwb,3,3
0859decd-5062-ee9c-a51f-575538d091f1,Toby20250316_.nwb,3,4
0859decd-5062-ee9c-a51f-575538d091f1,Toby20250316_.nwb,3,5
3700aa19-2c40-9e11-eac2-18bbf869340b,Toby20250316_.nwb,3,1
3700aa19-2c40-9e11-eac2-18bbf869340b,Toby20250316_.nwb,3,2
3700aa19-2c40-9e11-eac2-18bbf869340b,Toby20250316_.nwb,3,3
3700aa19-2c40-9e11-eac2-18bbf869340b,Toby20250316_.nwb,3,4
3700aa19-2c40-9e11-eac2-18bbf869340b,Toby20250316_.nwb,3,5


pos_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),block the block number within the epoch,analysis_file_name name of the file,hex_assignment_object_id
4eeb7432-969b-f856-3cfb-dd4456754ac4,Toby20250316_.nwb,1,1,Toby20250316_DJMR3XJ5R5.nwb,8dd3eb76-74cc-4a87-b0e4-14e86ff64775
4eeb7432-969b-f856-3cfb-dd4456754ac4,Toby20250316_.nwb,1,2,Toby20250316_R9Q6PYRISX.nwb,ef05f3b5-de59-401a-bfa0-324d3d83aa21
713b3445-c1da-79f1-4b8e-c5760e604d20,Toby20250316_.nwb,1,1,Toby20250316_GDCHALINXK.nwb,87bd1109-8dc0-410a-ab5d-88b6e42d7b90
713b3445-c1da-79f1-4b8e-c5760e604d20,Toby20250316_.nwb,1,2,Toby20250316_I7ZS91WHKO.nwb,17b13ac9-9d66-4f95-8ff6-5e47a16d07bc
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,1,IM-1478_20220725_UMT382KOJM.nwb,4241cba2-6d2a-400d-aaf9-6a376759086d
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,2,IM-1478_20220725_RQUTK6XGIZ.nwb,a312723d-a4d8-4b78-b8ba-b156c10629ab
d9340e97-103b-099a-ca4d-96f6d9340cdd,IM-1478_20220725_.nwb,0,3,IM-1478_20220725_144DT22AR2.nwb,12515ee6-c24b-44da-943f-5a062d4dbe6d
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,1,IM-1478_20220725_SW6U1R3V8V.nwb,5bbe3ae9-8cc9-456f-8340-42c330927523
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,2,IM-1478_20220725_LRQW6XIAWB.nwb,42974c06-09fd-4d3e-9655-becb79314cf7
eab73e2c-358d-5657-3cc7-abd6c3f8ff85,IM-1478_20220725_.nwb,0,3,IM-1478_20220725_PVVCPR6FEJ.nwb,d7daebdd-4970-47b9-8c41-04ed84bb9746
